In [ ]:
# 將 training data 的 pickle 檔載入
import pickle
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import monpa as mp
import numpy as np
from tqdm import tqdm #進度條
mp.use_gpu(True)

train_df = pickle.load(open("./monpa_train.pickle", "rb"))
lables = list(train_df["Lable"])
contents = list(train_df["Content"])
print(contents[1])
le = preprocessing.LabelEncoder()
le.fit(["tech", "edu", "sports", "health", "travel", "politics"])
encoded_lable = le.transform(lables)



def data_gen(cts):
    tg=['Dfa','Dfb','Nv','VA','VAC','VB','VC','VCL','VD','VE','VF','VG','VH','VHC','VI','VJ','VK','VL','Na','A','Nc','Ncd','ORG','PER']
    result=[]

    for content in tqdm(cts):
        chosen_words=[]
        
        psegs=mp.pseg_batch(content)[0]
        for pseg in psegs:
            try:
                if pseg[1] in tg:
                    chosen_words.append(pseg[0].strip())
            except:
                print(pseg)
        #print(chosen_words)
        result.append(" ".join(chosen_words))
    return(result)

contents = data_gen(contents)


#train_df.head()

In [1]:
# 將 training data 的 pickle 檔載入
import pickle
import pandas as pd
from sklearn import preprocessing
train_df = pickle.load(open("./monpa_train.pickle", "rb"))
lables = list(train_df["Lable"])
contents = list(train_df["Content"])

le = preprocessing.LabelEncoder()
le.fit(["tech", "edu", "sports", "health", "travel", "politics"])
encoded_lable = le.transform(lables)

train_df.head()
# test_df.head()
# encoded_lable

Lable                                            Content
0    tech  工研院 撒 專利網   對準 三星   練兵 七 年 ， 工研院 終於 出手 撒網 ， 首 ...
1     edu  披值 星帶 成 歷史   國軍 7 / 7 起 取消     軍 中 掛值 星帶 操課 將 ...
2  sports  倫敦 沒 收 刀械 槍枝     回收 充當 奧運 設施 建材       （ 法新社 倫敦...
3  sports  NBA 》 杜蘭特 比下 布萊恩   雷霆 擊敗 湖人   湖人 客場 挑戰 雷霆 ， 布萊...
4  health  誆 治 病 ！ 500 人 植入 整 條 臍帶   警 追上 游  常 聽到 的 臍帶血 移...

In [2]:
# 對載入後的 training pickle 對模型進行訓練
from sklearn.naive_bayes import ComplementNB # ComplementNB/MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
model = ComplementNB() # ComplementNB/MultinomialNB
tfidf_vectorizer = TfidfVectorizer()
model.fit(tfidf_vectorizer.fit_transform(contents), 
         encoded_lable)

ComplementNB()

In [ ]:
# 將 test data 的 pickle 檔載入
test_df = pickle.load(open("./monpa_test.pickle", "rb"))
test_id = list(test_df["ID"])
test_contents = list(test_df["Content"])
test_df.head()

In [ ]:
# 對載入後的 test pickle 以訓練過之後的模型進行預測
pred = model.predict(
    tfidf_vectorizer.transform(test_contents))
pred_label = list(le.inverse_transform(pred))
#將標籤序列加入預測資料之後
test_df.insert(len(test_df.columns), "Label", pred_label)

In [ ]:
# 刪除不要的欄位並產生結果檔
test_df = test_df.drop(["Content"], axis = 1)
test_df.to_csv("submission.csv", index = False)